In [1]:
import requests  
from bs4 import BeautifulSoup  
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import unicodedata
import re
import json
import numpy as np

In [2]:
#Updated State Law Source
#Original article: https://www.stateside.com/blog/2020-state-and-local-government-responses-covid-19
#iframe containing html to parse
r = requests.get('https://docs.google.com/spreadsheets/u/0/d/e/2PACX-1vQgu1AdYOZ0UurffV0b6DQEuNmr46lW0I9TvfpR1sZHnIZH3TBmIVbUdrdk_T6J5vbMKv9KUzgrRBIV/pubhtml/sheet?headers=false&gid=0')
soup = BeautifulSoup(r.text, 'html.parser')  
results = soup.find_all('tbody')[0].find_all('tr')
del results[1]
text_output = []
for result in results:
    for i in result.find_all('td'):
        text_output.append(i.text)

states = text_output[0::5]
type_of_actions = text_output[1::5]
actions = text_output[2::5]
summarys = text_output[3::5]
statuss = text_output[4::5]

records = []
for state,type_of_action,action,summary,status in zip(states,type_of_actions,actions,summarys,statuss):
    records.append((state,type_of_action,action,summary,status))
del records[0]

df = pd.DataFrame(records, columns =['StateName', 'Type of Action','Action','Summary','Status']) 
df['LawNumber'] = df.sort_values(['StateName','Type of Action'], ascending=True) \
             .groupby(['StateName']) \
             .cumcount() + 1
df.to_excel("stateside_laws.xlsx",header=False,encoding="utf-8-sig",index=False) 

In [3]:
#Updated nytimes US stay @ home order coverage
r = requests.get('https://www.nytimes.com/interactive/2020/us/states-reopen-map-coronavirus.html')
soup = BeautifulSoup(r.text, 'html.parser') 


records = []
#Reopened Group
results = soup.find_all('div',{'class':'g-list-container g-cat-reopened'})[0].find_all('div',{'class':'g-inner-box'})
for result in results:
    state_open_details = []
    state_details = result.find('div',{'class':'g-date-details'}).text.strip()
    state_details = state_details.replace(u'\n', u' ')
    if result.find('span',{'class':'g-link'}):
        state_link = result.find('a').get('href')
    else:
        state_link = None
    if result.find('span',{'class':'g-text'}):
        state_facts = result.find('span',{'class':'g-text'}).text
    else:
        state_facts = None
   
    state_name_tag = result.find('div',{'class':'g-stateCaseChartShell'})
    state_name = state_name_tag['data-state']
    status = 'Reopened'
    if result.find('div',{'class':'g-details-wrap-outer'}):
        state_bullets = result.find('div',{'class':'g-details-wrap-outer'}).text.strip()
        state_bullets = state_bullets.replace(u'Reopened',u'')
    else:
        state_bullets = None
    records.append([state_name,state_details,state_link,status,state_facts,state_bullets])
#Reopening Group
results = soup.find_all('div',{'class':'g-list-container g-cat-forward'})[0].find_all('div',{'class':'g-inner-box'})
for result in results:
    state_open_details = []
    state_details = result.find('div',{'class':'g-date-details'}).text.strip()
    state_details = state_details.replace(u'\n', u' ')
    if result.find('span',{'class':'g-link'}):
        state_link = result.find('a').get('href')
    else:
        state_link = None
    if result.find('span',{'class':'g-text'}):
        state_facts = result.find('span',{'class':'g-text'}).text
    else:
        state_facts = None
   
    state_name_tag = result.find('div',{'class':'g-stateCaseChartShell'})
    state_name = state_name_tag['data-state']
    status = 'Reopening'
    if result.find('div',{'class':'g-details-wrap-outer'}):
        state_bullets = result.find('div',{'class':'g-details-wrap-outer'}).text.strip()
        state_bullets = state_bullets.replace(u'Reopened',u'')
    else:
        state_bullets = None
    records.append([state_name,state_details,state_link,status,state_facts,state_bullets])
#Pausing Group
results = soup.find_all('div',{'class':'g-list-container g-cat-pausing'})[0].find_all('div',{'class':'g-inner-box'})
for result in results:
    state_open_details = []
    state_details = result.find('div',{'class':'g-date-details'}).text.strip()
    state_details = state_details.replace(u'\n', u' ')
    if result.find('span',{'class':'g-link'}):
        state_link = result.find('a').get('href')
    else:
        state_link = None
    if result.find('span',{'class':'g-text'}):
        state_facts = result.find('span',{'class':'g-text'}).text
    else:
        state_facts = None
    state_name_tag = result.find('div',{'class':'g-stateCaseChartShell'})
    state_name = state_name_tag['data-state']
    status = 'Pausing'
    if result.find('div',{'class':'g-details-wrap-outer'}):
        state_bullets = result.find('div',{'class':'g-details-wrap-outer'}).text.strip()
        state_bullets = state_bullets.replace(u'Reopened',u'')
    else:
        state_bullets = None
    records.append([state_name,state_details,state_link,status,state_facts,state_bullets])
#Reversing Group
results = soup.find_all('div',{'class':'g-list-container g-cat-reversing'})[0].find_all('div',{'class':'g-inner-box'})
for result in results:
    state_open_details = []
    state_details = result.find('div',{'class':'g-date-details'}).text.strip()
    state_details = state_details.replace(u'\n', u' ')
    if result.find('span',{'class':'g-link'}):
        state_link = result.find('a').get('href')
    else:
        state_link = None
    if result.find('span',{'class':'g-text'}):
        state_facts = result.find('span',{'class':'g-text'}).text
    else:
        state_facts = None
    state_name_tag = result.find('div',{'class':'g-stateCaseChartShell'})
    state_name = state_name_tag['data-state']
    status = 'Reversing'
    if result.find('div',{'class':'g-details-wrap-outer'}):
        state_bullets = result.find('div',{'class':'g-details-wrap-outer'}).text.strip()
        state_bullets = state_bullets.replace(u'Reopened',u'')
    else:
        state_bullets = None
    records.append([state_name,state_details,state_link,status,state_facts,state_bullets])

df = pd.DataFrame(records, columns =['StateName', 'State Details','State Link','Status','State Blurb','State Bullets']) 
df.to_excel("NYTimes_Updated_StateStayAtHomeDetails.xlsx",header=True,encoding="utf-8-sig",index=False) 

In [29]:
#Updated school enrollment for Fall 2020 School Year
#Original article is here: https://www.edweek.org/ew/section/multimedia/map-covid-19-schools-open-closed.html
#Below iframe contains javascript with dictionaries for school reopening
url = 'https://flo.uri.sh/visualisation/2451841/embed?auto=1'

html_data = requests.get(url).text
data = re.search(r'_Flourish_data = (\{.*?\});', html_data).group(1)

data = json.loads(data)
records = []
for row in data['rows']:
    state_name = row['columns'][0].strip()
    public_school_enrollment = row['columns'][1].strip()
    school_details = row['columns'][2].strip()
    school_link = BeautifulSoup(row['columns'][3],'html.parser').find('a').get('href')
    records.append((state_name,public_school_enrollment,school_details,school_link))
df = pd.DataFrame(records, columns =['StateName', 'public_school_enrollment','SchoolDetails','SchoolLink']) 
df.to_excel("fall_school_closure_status.xlsx",header=True,encoding="utf-8-sig",index=False) 

In [67]:
#Updated school reopening plans for Fall 2020 School Year
#Original article is here: https://www.edweek.org/ew/section/multimedia/school-districts-reopening-plans-a-snapshot.html
#Below iframe contains javascript with dictionaries for school reopening
url = 'https://flo.uri.sh/visualisation/3186154/embed'
state_names = pd.read_excel (r'StateAbbreviationNames.xlsx')
state_list = state_names.set_index('Abbreviation')
html_data = requests.get(url).text
data = re.search(r'_Flourish_data = (\{.*?\});', html_data).group(1)

data = json.loads(data)

records = []
for row in data['rows']:
#     print('{:<55}{}'.format(*map(str.strip, row['columns'])))
    state_name = row['columns'][1].strip()
    public_school_enrollment = row['columns'][2].strip()
    last_verified = row['columns'][3].strip()
    district_reopening_plans = row['columns'][4].strip()
    start_date = row['columns'][5].strip()
    school_link = BeautifulSoup(row['columns'][6],'html.parser').find('a').get('href')
    records.append((state_name,public_school_enrollment,last_verified,district_reopening_plans,start_date,school_link))

df = pd.DataFrame(records, columns =['state_name', 'public_school_enrollment','last_verified','district_reopening_plans','startdate','school_link']) 

df.startdate = pd.to_datetime(df.startdate).values.astype(np.int64)

df = pd.DataFrame(pd.to_datetime(df.groupby('state_name').mean().startdate))
result = pd.merge(df, state_list,left_index=True,right_index=True)
result.to_excel("average_school_start_dates.xlsx",header=True,encoding="utf-8-sig",index=True) 

In [5]:
#Updated NYTimes case information

url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv'
df = pd.read_csv(url, error_bad_lines=False)
df = df.melt(id_vars=['date', 'state','fips'])
df = df.rename(columns={'variable': 'CaseType','value':'Cases'})
df = df.replace({'CaseType':{'cases':'Confirmed','deaths':'Deaths'}})
df.sort_values(by=['state','fips','CaseType','date'], inplace=True)
df["Difference"] = df.Cases.diff()
mask = df.CaseType != df.CaseType.shift(1)
df["Difference"][mask] = np.nan
df.to_excel("COVID-19 Cases.xlsx",header=True,encoding="utf-8-sig",index=False) 